<a href="https://colab.research.google.com/github/kushagra8881/multimodelllm/blob/main/work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers datasets peft trl openai-clip bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_c

In [ ]:
import torch
import os
import json
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer
# if you using google collab use it

from google.colab import drive
import clip
from PIL import Image
import pandas as pd

In [ ]:
drive.mount('/content/drive')

In [ ]:
base_model = "NousResearch/Llama-2-7b-chat-hf"
new_model_path = "/content/drive/MyDrive/llama-2-7b-chat-updated"

In [ ]:
with open('/content/drive/MyDrive/fine/data.json', 'r') as f:
    data = json.load(f)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_clip, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
def image_to_label(image_path, labels):
    # Preprocess the image
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

    # Encode the image
    with torch.no_grad():
        image_features = model_clip.encode_image(image)

    # Tokenize and encode the labels
    text_inputs = clip.tokenize(labels).to(device)
    with torch.no_grad():
        text_features = model_clip.encode_text(text_inputs)

    # Calculate the similarity between image features and text features
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

    # Get the label with the highest similarity
    best_match_idx = similarity.argmax(dim=-1).item()
    best_label = labels[best_match_idx]

    return best_label

In [ ]:
data

In [ ]:
labels=["a dog palying in a park" , "a dog dancing in the park"]

In [ ]:
image_directory = '/content/drive/MyDrive/fine/'

# Process the data
processed_data = []
for item in data:
    image_path = image_directory + item['image']
    label = image_to_label(image_path, labels=labels)
    for conversation in item['conversations']:
        if conversation['from'] == 'human':
            prompt = conversation['value'].replace("<image>","an image of "+ label)
        else:
            response = conversation['value']
            processed_data.append({"prompt": prompt, "response": response})

# Convert to Hugging Face dataset
df = pd.DataFrame(processed_data)
dataset = Dataset.from_pandas(df)

# Print the processed data for verification
print(processed_data)

In [ ]:
compute_dtype = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="prompt",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)


In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model_path)
trainer.tokenizer.save_pretrained(new_model_path)

In [ ]:
from tensorboard import notebook
log_dir = "results/runs"
notebook.start("--logdir {} --port 4000".format(log_dir))